In [1]:
import numpy as np
import pandas as pd
import boto3

In [2]:
# Let's use Amazon S3
# The Access Key ID and Secret Access Key are in ~/.aws/credentials
# Region is set in ~/.aws/config
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

athena-query-bucket-testda24
aws-glue-assets-936480984004-eu-north-1
hacksprint-2023
temp-transformation-rs
vehicle-bucket-alarm


In [4]:
# Print the file names in the bucket
revolvos_bucket = s3.Bucket('hacksprint-2023')
for object in revolvos_bucket.objects.all():
    print(object.key)

Telematics Data Dictionary 2023_02_14.xlsx
Telematics Data Dictionary 2023_02_14.xlsx - SNAP_SHOT_ITEM (1).csv
Vehicle_Fault.csv
Vehicle_Fault_Alarm.csv
Vehicle_Fault_Alarm_Item.csv
Vehicle_Location.csv
Vehicle_Specs.csv


In [ ]:
%%time
# Load one of the files into a dataframe
obj = s3.Object('hacksprint-2023', 'Vehicle_Fault.csv').get()
df = pd.read_csv(obj['Body'])

In [5]:
%%time
# Load from the local CSV file instead.
df = pd.read_csv('../S3 original bucket/Vehicle_Fault.csv')

CPU times: user 29.1 s, sys: 3.75 s, total: 32.9 s
Wall time: 33.2 s


In [6]:
print(df.dtypes)

ORBCOMM_MESSAGE_ID           int64
ORDER_ID                     int64
FAULT_DATE_TIME             object
FAULT_OCCUR_LONGITUDE      float64
FAULT_OCCUR_LATITUDE       float64
FAULT_COUNT                  int64
FMI                          int64
FAULT_TYPE                 float64
FAULT_ACTIVE                object
EXPANSION_DIAG_CODE        float64
VEHICLE_ID                   int64
COMPANY_ID                   int64
MESSAGE_DATE_TIME           object
VEHICLE_MESSAGE_ID           int64
COMPONENT_TYPE              object
ECU_TYPE                    object
ECU_ID                       int64
COMPONENT_ID                 int64
MSW                        float64
DST1                       float64
DST2                       float64
FAULT_FILTER_ID            float64
MALFUNCTION_LAMP_STATUS     object
RED_LAMP_STATUS             object
YELLOW_LAMP_STATUS          object
PROTECT_LAMP_STATUS         object
dtype: object


In [7]:
print(df.head())

   ORBCOMM_MESSAGE_ID  ORDER_ID      FAULT_DATE_TIME  FAULT_OCCUR_LONGITUDE  \
0          3968769921         1  7/1/2022 2:47:04 AM                    NaN   
1          3968768985         1  7/1/2022 2:46:58 AM                    NaN   
2          3968784712         1  7/1/2022 3:03:07 AM                    NaN   
3          3968783877         1  7/1/2022 3:03:02 AM                    NaN   
4          3971138033         1  7/2/2022 4:13:19 PM                    NaN   

   FAULT_OCCUR_LATITUDE  FAULT_COUNT  FMI  FAULT_TYPE FAULT_ACTIVE  \
0                   NaN          103   14         NaN            N   
1                   NaN          103   14         NaN            Y   
2                   NaN          104   14         NaN            N   
3                   NaN          104   14         NaN            Y   
4                   NaN          105   14         NaN            Y   

   EXPANSION_DIAG_CODE  ...  ECU_ID  COMPONENT_ID         MSW  DST1 DST2  \
0                  NaN  ... 

In [8]:
print(df.shape)

(25020193, 26)


Insert magical cleaning...

In [ ]:
%%time
# Send the cleaned dataframe content back to a new CSV file on the S3 bucket
csv_clean = df.to_csv(index=False)
_ = s3.Object('hacksprint-2023', 'Vehicle_Fault-clean.csv').put(Body=csv_clean)